In [1]:
import sys
sys.path.append("../src")

import requests

from tonpy import LiteClient, Cell, get_block_info, BlockId, BlockIdExt, Address, Emulator, begin_cell
from tonpy.utils.shard_account import shard_is_ancestor, shard_child
from tonpy.autogen.block import Account, Transaction, Block, BlockInfo, BlockExtra, MessageAny, AccountState

from tonpy.autogen.block import Transaction
from deepdiff import DeepDiff

# Ignore some logs from ton monorepo
from tonpy.libs.python_ton import globalSetVerbosity
globalSetVerbosity(9)

In [2]:
# Example hashes:
#   27 txs in 1 block: 184C0C0EF74E279E63B363FCC81168AEB4C3769E4991FAF6982A5E232EE88541
#   After merge: 7A074921055E014CE6B5E7C929B12D9789F5419E771F1BD98C379A57DA15064A
#   TickTock: F8EAC78DA7005B67B4A473CB77CF90ACD4A31B883E0E6C2806ADB4C0D2B3E616

In [3]:
tx_hash = "79124F21171D3227172246F8257F825F7B9B43436F5249FA832E954557A9FF31"

In [4]:
def get_tx_lt(tx_hash):
    query = '''query{transactions(hash: "%s"){lt,address,workchain,shard}}''' % tx_hash

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['transactions'][0]
    return int(data['lt']), int(data['workchain']), str(data['address']), str(data['shard'])

In [5]:
# You can not trust this data from dton.io, but IF it's correct it'll be prooved by `get_transactions` later
tx_lt, wc, address, shard = get_tx_lt(tx_hash)
block_lt = tx_lt - tx_lt % 1000000

tx_account = Address(f"{wc}:{address}")

In [6]:
# Get random LiteServer
lc = LiteClient.get_one()

# When you get transaction - you automatically proof validness by tx_hash
transactions = lc.get_transactions(tx_account, tx_lt, tx_hash, 16).transactions

# We need to get all TXs in this block
whitelist_txs = [transactions[0]]
get_state_from = None
cur_index = 0

if len(transactions) > 1:
    done = transactions[0].blkid != transactions[1].blkid
else:
    done = True
    
while not done:
    cur_index += 1
    if cur_index > len(transactions) - 1:
        # Get new 16txs to load
        transactions.extend(lc.get_transactions(tx_account, 
                                          whitelist_txs[-1].prev_trans_lt, 
                                          whitelist_txs[-1].prev_trans_hash, 16).transactions)
    if cur_index > len(transactions):
        break
    
    if whitelist_txs[-1].blkid == transactions[cur_index].blkid:
        whitelist_txs.append(transactions[cur_index])
    else:
        done = True
        

[ 4][t 1][1706387988.527050018][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 132
[ 4][t 1][1706387988.566128254][BufferedFd.h:216][!outconn]	Flush write: +456B[left:0B]
[ 4][t 1][1706387988.614362240][BufferedFd.h:207][!outconn]	Flush read: +68B[total:68B]
[ 4][t 1][1706387988.614414215][adnl-ext-connection.cpp:80][!outconn]	len=64
[ 4][t 1][1706387988.614447117][adnl-ext-connection.cpp:163][!outconn]	received packet of size 64
[ 4][t 1][1706387988.628312111][BufferedFd.h:207][!outconn]	Flush read: +832B[total:832B]
[ 4][t 1][1706387988.628370047][adnl-ext-connection.cpp:80][!outconn]	len=828
[ 4][t 1][1706387988.628450155][adnl-ext-connection.cpp:163][!outconn]	received packet of size 828


In [7]:
# If there >1 TX in block - we need to emulate all of them before our one
print("To emulate, before our TX: ", len(whitelist_txs) - 1)

whitelist_txs = list(reversed(whitelist_txs))

To emulate, before our TX:  0


[ 4][t 1][1706213319.428908110][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213319.429810047][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213319.470447302][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213319.470494032][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213319.470525026][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213329.471795082][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213329.472608089][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213329.521200895][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213329.521269083][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213329.521311998][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213339.521765232][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706213500.328280210][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213500.329178333][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213500.370294094][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213500.370342016][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213500.370365143][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213510.371222019][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213510.371842146][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213510.413831949][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213510.413932085][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213510.413976192][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213520.414916277][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706213681.246286154][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213681.246886015][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213681.302373886][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213681.302493095][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213681.302529097][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213691.303816080][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213691.304183245][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213691.348114967][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213691.348231077][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213691.348273039][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213701.348647118][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706213862.212167978][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213862.212829113][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213862.258404255][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213862.258458138][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213862.258491039][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213872.259514332][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706213872.260750294][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706213872.309288025][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706213872.309422255][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706213872.309476137][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706213882.310661316][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

...

[ 4][t 1][1706214767.252169132][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706214767.252619028][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706214767.300602913][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706214767.300699949][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706214767.300755024][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706214777.301412106][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706214777.301664114][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706214777.357612133][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706214777.357661963][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706214777.357685089][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706214787.360038042][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706214948.194792986][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706214948.195010185][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706214948.241508245][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706214948.241535187][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706214948.241799116][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706214958.243074179][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706214958.243371010][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706214958.290656090][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706214958.290714979][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706214958.290755272][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706214968.292558193][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706215129.127448082][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706215129.127606153][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706215129.186922073][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706215129.186948299][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706215129.186960936][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706215139.188255072][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706215139.188532114][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706215139.241165161][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706215139.241192102][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706215139.241224051][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706215149.242341995][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706215310.018325090][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706215310.018529177][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706215310.061498880][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706215310.061547041][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706215310.061578035][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706215320.062556028][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706215320.062695026][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706215320.104762316][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706215320.104866266][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706215320.104915142][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706215330.106333017][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706215490.872324228][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706215490.873111248][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706215490.912023067][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706215490.912070990][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706215490.912186146][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706215500.913520336][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706215500.914501190][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706215500.972108126][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706215500.972169161][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706215500.972204208][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706215510.973526239][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

In [8]:
# Get block with prev state
# WARNING: this block is not proofed
state_block = whitelist_txs[0].blkid
current_block = lc.get_block_header(state_block)
print("Current block: ", current_block.blk_id, "\n")

# We need to get master block, so we can download libs and config
if wc != -1:
    # tlb python autogen
    block = Block().cell_unpack(current_block.virt_blk_root)
    block_info = BlockInfo().cell_unpack(block.info, True)
    
    # get master block id from WC
    master = BlockIdExt(id_=BlockId(-1, 0x8000000000000000, block_info.master_ref.master.seq_no), 
          file_hash=int(block_info.master_ref.master.file_hash, 2), 
          root_hash=int(block_info.master_ref.master.root_hash, 2))
else:
    master = current_block.blk_id

print("Master block: ", master, "\n")


Current block:  (0,4000000000000000,40738211):8E6340FED0A18FE10CD5E33771AAA48B30C6DC0E09AD565B21B4CE1A1EFEE66D:3E17F4354C2BE4C60C45D81915A6E3488C1D8749519D09AE3530D03908C2E89F 

Master block:  (-1,8000000000000000,34876223):827F6FDB7148D786DF1E69F01DD509D7F55FCFAEFA126336B75B1F56E5BB310E:E5C2F2184CC82C5F9D744340DAABD544CAF52575820A43CA1472D18DB8689788 



[ 4][t 1][1706387990.729840994][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 136
[ 4][t 1][1706387990.730187178][BufferedFd.h:216][!outconn]	Flush write: +204B[left:0B]
[ 4][t 1][1706387990.802373886][BufferedFd.h:207][!outconn]	Flush read: +620B[total:620B]
[ 4][t 1][1706387990.802451134][adnl-ext-connection.cpp:80][!outconn]	len=616
[ 4][t 1][1706387990.802512884][adnl-ext-connection.cpp:163][!outconn]	received packet of size 616


In [9]:
# Need to get rand_seed for block and get prev block with account state
current_full_block = lc.get_block(current_block.blk_id)

# It's stored in blockExtra
block = Block().cell_unpack(current_full_block)
block_info = BlockInfo().cell_unpack(block.info, True)
block_extra = BlockExtra().cell_unpack(block.extra, False)

rand_seed = int(block_extra.rand_seed, 2)

[ 4][t 1][1706387991.477733135][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 132
[ 4][t 1][1706387991.477903128][BufferedFd.h:216][!outconn]	Flush write: +200B[left:0B]
[ 4][t 1][1706387991.670014143][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:2576B]
[ 4][t 1][1706387991.670061111][adnl-ext-connection.cpp:80][!outconn]	len=235764
[ 4][t 1][1706387991.670747042][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:5148B]
[ 4][t 1][1706387991.671340227][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:6436B]
[ 4][t 1][1706387991.671556234][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:9012B]
[ 4][t 1][1706387991.672918081][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:11KB]
[ 4][t 1][1706387991.673050165][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:12KB]
[ 4][t 1][1706387991.754212141][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:13KB]
[ 4][t 1][1706387991.840507030][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:15KB]
[ 

In [10]:
# To get prev block with account state - need to calculate correct shard (with account) if block is after merge
# If not after merge - just get prev seqno
if block_info.after_merge:    
    left = block_info.prev_ref.prev1
    left_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, True), left.seq_no),
                           root_hash=int(left.root_hash,2), file_hash=int(left.file_hash, 2))
    
    right = block_info.prev_ref.prev2
    right_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, False), right.seq_no),
                           root_hash=int(right.root_hash,2), file_hash=int(right.file_hash, 2))
    
    if shard_is_ancestor(left_shard.id.shard, tx_account.shard_prefix(60)):
        state_block = left_shard
    else:
        state_block = right_shard
else:
    current = block_info.prev_ref.prev
    state_block = BlockIdExt(BlockId(wc, current_block.blk_id.id.shard, current.seq_no),
                           root_hash=int(current.root_hash,2), file_hash=int(current.file_hash, 2))

.061654091][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:105KB]
[ 4][t 1][1706387992.062199116][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:106KB]
[ 4][t 1][1706387992.066257000][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:108KB]
[ 4][t 1][1706387992.067146063][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:109KB]
[ 4][t 1][1706387992.068171024][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:110KB]
[ 4][t 1][1706387992.069397211][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:111KB]
[ 4][t 1][1706387992.069983006][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:113KB]
[ 4][t 1][1706387992.071155071][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:114KB]
[ 4][t 1][1706387992.072309017][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:115KB]
[ 4][t 1][1706387992.073096991][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:116KB]
[ 4][t 1][1706387992.073992014][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:118KB]
[ 4][t 1

In [11]:
print("State block: ", state_block, "\n")

State block:  (0,4000000000000000,40738210):7EEA03CE81616915252E4537BB0D647125487C19303411153C0A10ABCE73D454:FDBCFFF83196C45CA1F3E51D2CB121A7D474201DCB54D1DA31246F2D65485741 



In [13]:
# Load account state from prev block
account_state = lc.get_account_state(f"{wc}:{address}", state_block)

if not account_state.root.is_null():
    # Convert answer to AccountShardState
    account_state = begin_cell() \
        .store_ref(account_state.root) \
        .store_uint(int(account_state.last_trans_hash, 16), 256) \
        .store_uint(account_state.last_trans_lt, 64).end_cell()
else:
     account_state = begin_cell() \
        .store_ref(begin_cell().store_uint(0, 1).end_cell()) \
        .store_uint(int(account_state.last_trans_hash, 16), 256) \
        .store_uint(account_state.last_trans_lt, 64) \
        .end_cell()

[ 4][t 1][1706388001.115023136][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 168
[ 4][t 1][1706388001.115262985][BufferedFd.h:216][!outconn]	Flush write: +236B[left:0B]
[ 4][t 1][1706388001.162146091][BufferedFd.h:207][!outconn]	Flush read: +320B[total:320B]
[ 4][t 1][1706388001.162184954][adnl-ext-connection.cpp:80][!outconn]	len=316
[ 4][t 1][1706388001.162211180][adnl-ext-connection.cpp:163][!outconn]	received packet of size 316


RuntimeError: cannot load state for (0,4000000000000000,40738210):7EEA03CE81616915252E4537BB0D647125487C19303411153C0A10ABCE73D454:FDBCFFF83196C45CA1F3E51D2CB121A7D474201DCB54D1DA31246F2D65485741 : state already gc'd

[ 4][t 1][1706388011.165027142][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706388011.165292025][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706388011.201904297][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706388011.201941013][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706388011.201965094][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706388021.203050137][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706388021.203279257][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706388021.365976095][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706388021.366023064][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706388021.366089106][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706388031.367281914][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

In [ ]:
# Load config of prev key block of MC block where TX was
# WARNING: this key block have no proof
key_block, config = lc.get_config_all(master, from_not_trusted_keyblock=True)

In [ ]:
print("Latest key block: ", key_block)

In [ ]:
# for emulation you need to get 16 latest mc blocks + 1 key block
old_mc_blocks = []
cur_seqno = master.id.seqno

for i in range(16):
    cur_seqno -= 1
    old_mc_blocks.append(lc.lookup_block(BlockId(-1, 0x8000000000000000, cur_seqno)).blk_id)

In [ ]:
prev_block_data = [
    [i.to_data() for i in old_mc_blocks],
    key_block.to_data()
]

In [ ]:
# Here we need to get all public libraries (todo: rewrite on new LS API if it'll be published)
# Alternative: deep_library_search 
def get_public_libs(block_hash):
    query = '''query{blocks(root_hash: "%s"){libs_hash}}''' % block_hash

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['blocks'][0]
    return data['libs_hash']

In [ ]:
public_libs = get_public_libs(master.root_hash)

print("Found: ", len(public_libs), " public libs")

libs_data = lc.get_libraries(public_libs)

In [ ]:
# Now create emulator with all needed info
em = Emulator(config)

em.set_rand_seed(rand_seed)
em.set_prev_blocks_info(prev_block_data)
em.set_libs(libs_data)

In [ ]:
account_state.get_hash()

In [ ]:
def get_diff(tx1, tx2):
    tx1_tlb = Transaction()
    tx1_tlb = tx1_tlb.cell_unpack(tx1, True).dump()

    tx2_tlb = Transaction()
    tx2_tlb = tx2_tlb.cell_unpack(tx2, True).dump()

    diff = DeepDiff(tx1_tlb, tx2_tlb).to_dict()

    address = tx1_tlb['account_addr']
    del tx1_tlb
    del tx2_tlb

    return diff, address


In [ ]:
initial_account_state = account_state

In [ ]:
account_state = initial_account_state

In [ ]:
for tx in whitelist_txs:
    # Get current transaction LT & now
    current_tx = tx.transaction

    tx_tlb = Transaction()
    tx_tlb = tx_tlb.cell_unpack(current_tx, True)

    lt = tx_tlb.lt
    now = tx_tlb.now
    
    # Get current trasaction in_msg
    current_tx_cs = current_tx.begin_parse()
    tmp = current_tx_cs.load_ref(as_cs=True)

    if tmp.load_bool():
        in_msg = tmp.load_ref()
    else:
        # TICK TOCK
        in_msg = None

    if in_msg is None:
        success = em.emulate_tick_tock_transaction(
                    account_state,
                    tx_tlb.description.is_tock,
                    now,
                    lt
                )
    else:
        # Emulate
        success = em.emulate_transaction(
            account_state,
            in_msg,
            now,
            lt
        )
        
    assert success
    
    # Emulation transaction equal current transaction
    if em.transaction.get_hash() != current_tx.get_hash():
        print(get_diff(current_tx, em.transaction.to_cell()))
        raise ValueError
    else:
        print(f"\tEmulate {current_tx.get_hash()} success\n\n")
    
    # Update account state, go to next transaction
    account_state = em.account.to_cell()
    print(account_state.dump_as_tlb("ShardAccount"))